In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import tqdm 
from typing import List 

import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW

In [2]:
from tqdm import tqdm

In [13]:
# Using Todd library to implement scorers

from Todd import (
    ScorerType, 
    MahalanobisScorer, 
    CosineProjectionScorer, 
    DataDepthScorer, 
)

from toddbenchmark.classification_datasets import (prep_model, prep_dataset, load_sst2)
from toddbenchmark.utils_classification import evaluate_dataloader


In [4]:
model, tokenizer = prep_model("echarlaix/bert-base-uncased-sst2-acc91.1-d37-hybrid")

In [5]:
config_sst2=  {"label": 2, "metric": "sst2", "keys": ("text", None)}
config_imdb = {"label": 2, "metric": "sst2", "keys": ("text", None)}

in_dataset = prep_dataset("sst2", config_sst2, tokenizer=tokenizer, train_max_size=0, validation_max_size=1000, test_max_size=100)
out_dataset = prep_dataset("imdb", config_imdb, tokenizer=tokenizer, train_max_size=1000, validation_max_size=1000,test_max_size=100)

Found cached dataset glue (/Users/joevincentgaltie/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset imdb (/Users/joevincentgaltie/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
in_val = in_dataset[1]
in_test = in_dataset[2]
out_test = out_dataset[2]


in_val_loader = DataLoader(in_val, shuffle=False, batch_size=4)
in_test_loader = DataLoader(in_test, shuffle=False, batch_size=4)
out_test_loader = DataLoader(out_test, shuffle=False, batch_size=4)

In [7]:
maha_detector = MahalanobisScorer(layers=[-1])
maha_detector.accumulated_embeddings

defaultdict(list, {})

In [8]:
type(maha_detector)

Todd.featuresscorers.MahalanobisScorer

In [9]:
in_test_loader

In [10]:
in_val

Dataset({
    features: ['text', 'labels'],
    num_rows: 872
})

In [11]:
def prepare_embeddings(scorer, loader):
    with torch.no_grad():
        for batch in tqdm(loader):
            inputs = tokenizer(
                batch["text"], padding=True, truncation=True, return_tensors="pt"
            )
            output = model(
                **inputs,
                #return_dict_in_generate=True,
                output_hidden_states=True,
                #output_scores=True,
            )
            output['encoder_hidden_states'] = output['hidden_states']

            scorer.accumulate(output,y=None)

prepare_embeddings(maha_detector, loader=in_val_loader)
maha_detector.fit()

100%|██████████| 218/218 [00:29<00:00,  7.36it/s]


In [53]:
def eval_loader(loader):
    with torch.no_grad():
        for batch in tqdm(loader):
            inputs = tokenizer(
                batch["text"], padding=True, truncation=True, return_tensors="pt"
            )
            output = model(
                **inputs,
                #return_dict_in_generate=True,
                output_hidden_states=True,
                #output_scores=True,
            )

            output['encoder_hidden_states'] = output['hidden_states']
            
            print(maha_detector(output))
            break


print(eval_loader(in_test_loader))
print(eval_loader(out_test_loader))

  0%|          | 0/25 [00:00<?, ?it/s]


tensor([419778.5000, 377216.8438, 325469.4375, 466846.4375])
None


  0%|          | 0/25 [00:01<?, ?it/s]

tensor([1185780.5000, 1372989.5000, 1780022.8750, 1078624.8750])
None


In [15]:
print("Evaluating on the in-distribution training set")

records_in_train = evaluate_dataloader(
    model,
    in_val_loader,
    tokenizer,
    maha_detector
)

Evaluating on the in-distribution training set


TypeError: 'MahalanobisScorer' object is not iterable

In [48]:
maha_detector.accumulated_embeddings

AttributeError: 'MahalanobisScorer' object has no attribute 'accumulated_embeddings'

In [ ]:
ev

In [69]:
 
inputs = tokenizer(
               next(iter(in_val_loader))["text"], padding=True, truncation=True, return_tensors="pt"
            )

output = model(
                **inputs,
                #return_dict_in_generate=True,
                output_hidden_states=True,
                #output_scores=True,
            )




In [70]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[-3.5209,  4.5214],
        [ 2.4283, -2.5945],
        [-3.1178,  3.9888],
        [-2.7755,  3.6769]], grad_fn=<AddmmBackward0>), hidden_states=(tensor([[[ 0.1560, -0.2660, -0.3172,  ..., -0.0048,  0.0145,  0.1522],
         [-0.4534, -0.0029, -0.0558,  ...,  0.7296,  0.9141,  0.1591],
         [-0.7016,  0.2778,  0.2457,  ...,  0.4870,  0.6775, -1.1272],
         ...,
         [ 0.1444, -0.2958, -0.2512,  ...,  0.1014, -0.3191,  0.0963],
         [-0.1005, -0.3642, -0.0352,  ..., -0.1397, -0.1140,  0.0108],
         [-0.0458, -0.4816, -0.0440,  ..., -0.3120, -0.2293, -0.0493]],

        [[ 0.1560, -0.2660, -0.3172,  ..., -0.0048,  0.0145,  0.1522],
         [-0.6175, -0.2491,  0.1479,  ...,  0.3129,  0.4923,  0.2938],
         [ 0.2888,  0.8513,  0.1758,  ..., -0.0930,  0.8163, -0.0227],
         ...,
         [ 0.1444, -0.2958, -0.2512,  ...,  0.1014, -0.3191,  0.0963],
         [-0.1005, -0.3642, -0.0352,  ..., -0.1397, -0.1140,  

In [67]:
from toddbenchmark.utils_classification import prepare_detectors
detectors: List[ScorerType] = [
    MahalanobisScorer(layers=[-1])]

detectors = prepare_detectors(detectors, model, in_val_loader, tokenizer)

AttributeError: 'SequenceClassifierOutput' object has no attribute 'sequences'